<a href="https://colab.research.google.com/github/l4zy0n3/Python/blob/master/India.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json, requests
import pandas as pd
import plotly.graph_objects as go
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as plt
import fbprophet
import datetime
import numpy as np

In [0]:
country = 'India'

In [0]:
timeseries = json.loads(requests.get('https://pomber.github.io/covid19/timeseries.json').text)

In [555]:
timeseries[country][0:5]
# all = timeseries[]

[{'confirmed': 0, 'date': '2020-1-22', 'deaths': 0, 'recovered': 0},
 {'confirmed': 0, 'date': '2020-1-23', 'deaths': 0, 'recovered': 0},
 {'confirmed': 0, 'date': '2020-1-24', 'deaths': 0, 'recovered': 0},
 {'confirmed': 0, 'date': '2020-1-25', 'deaths': 0, 'recovered': 0},
 {'confirmed': 0, 'date': '2020-1-26', 'deaths': 0, 'recovered': 0}]

In [0]:
df = pd.DataFrame.from_dict(timeseries[country])

In [557]:
df = df.set_index('date')
diff_data = df.diff()
diff_data.head()

,confirmed,deaths,recovered
date,,,
2020-1-22,NaN,NaN,NaN
2020-1-23,0.0,0.0,0.0
2020-1-24,0.0,0.0,0.0
2020-1-25,0.0,0.0,0.0
2020-1-26,0.0,0.0,0.0


In [558]:
fig = go.Figure(data=[
                      go.Scatter(name='Total cases', x=df.index, y=df['confirmed']),
                      go.Scatter(name='Deaths', x=df.index, y=df['deaths']),
                      go.Scatter(name='Recoveries', x=df.index, y=df['recovered'])
                      ])
fig.update_layout(yaxis_type="log")
fig.show()

In [559]:
df.corr()

,confirmed,deaths,recovered
confirmed,1.000000,0.985993,0.982610
deaths,0.985993,1.000000,0.980646
recovered,0.982610,0.980646,1.000000


In [560]:
fig = go.Figure(data=[
                      go.Scatter(name='Total cases', x=diff_data.index, y=diff_data['confirmed']),
                      go.Scatter(name='Deaths', x=diff_data.index, y=diff_data['deaths']),
                      go.Scatter(name='Recoveries', x=diff_data.index, y=diff_data['recovered'])
                      ])
fig.update_layout()
fig.show()

# Trying with log scaled data

In [561]:
df_log =  df.apply(np.log).replace([np.inf, -np.inf], np.nan).dropna(subset=["deaths", "confirmed", "recovered"], how="all")
df_log.tail()

,confirmed,deaths,recovered
date,,,
2020-3-25,6.487684,2.484907,3.761200
2020-3-26,6.588926,2.995732,3.806662
2020-3-27,6.787845,2.995732,4.290459
2020-3-28,6.894670,3.178054,4.430817
2020-3-29,6.931472,3.295837,4.553877


In [562]:
# deaths_df = diff_data.reset_index().filter(["date", "deaths"], axis = 1).rename( columns={ 'date': 'ds', 'deaths': 'y'})
# deaths_df
conf_df = df_log.reset_index().filter(["date", "confirmed"], axis = 1).rename( columns={ 'date': 'ds', 'confirmed': 'y'})
conf_df
mdl = fbprophet.Prophet( growth='linear')
# mdl.add_changepoints_to_plot()
mdl.fit(conf_df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [563]:
forecast = mdl.make_future_dataframe(periods=5, freq='D')
forecast = mdl.predict(forecast)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
60,2020-03-30,7.407821,6.963442,7.817732
61,2020-03-31,7.547261,7.116341,7.962946
62,2020-04-01,7.887910,7.459748,8.313864
63,2020-04-02,7.966425,7.550478,8.411693
64,2020-04-03,8.097601,7.680153,8.551792


In [564]:
# fig = mdl.plot( forecast, xlabel= 'Date', ylabel='Confirmed')
fig = plot_plotly(mdl, forecast)  # This returns a plotly Figure
# a = add_changepoints_to_plot(fig.gca(), mdl, forecast)
plt.iplot(fig)

In [0]:
forecast = forecast.set_index('ds') 
forecast = forecast[[ 'yhat', 'yhat_lower', 'yhat_upper']].apply(np.exp).apply(np.round)

In [566]:
forecast[30:]

,yhat,yhat_lower,yhat_upper
ds,,,
2020-02-29,6.0,4.0,9.0
2020-03-01,7.0,4.0,10.0
2020-03-02,9.0,6.0,14.0
2020-03-03,11.0,7.0,17.0
2020-03-04,15.0,10.0,23.0
2020-03-05,16.0,11.0,26.0
2020-03-06,18.0,12.0,28.0
2020-03-07,21.0,14.0,33.0
2020-03-08,26.0,17.0,40.0


In [0]:
df.index = pd.to_datetime(df.index)

In [568]:
forecast = pd.concat( [df, forecast.replace( np.nan, 0).astype('int64')])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [0]:
df = df.replace( np.nan, 0)

In [570]:
forecast

,confirmed,deaths,recovered,yhat,yhat_lower,yhat_upper
2020-01-22,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-23,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-24,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-25,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-26,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-03-30,NaN,NaN,NaN,1649.0,1057.0,2484.0
2020-03-31,NaN,NaN,NaN,1896.0,1232.0,2873.0
2020-04-01,NaN,NaN,NaN,2665.0,1737.0,4080.0
2020-04-02,NaN,NaN,NaN,2883.0,1902.0,4499.0


In [571]:
fig = go.Figure(data=[
                      go.Scatter(name='Upper limit', x=forecast.index, y=forecast['yhat_upper']),
                      go.Scatter(name='Confirmed Real', x=forecast.index, y=forecast['confirmed']),
                      go.Scatter(name='Confirmed Predicted', x=forecast.index, y=forecast['yhat']),
                      go.Scatter(name='Lower limit', x=forecast.index, y=forecast['yhat_lower'])
                      ])
fig.update_layout()
fig.show()

# Trying with unscaled data

In [572]:
# deaths_df = diff_data.reset_index().filter(["date", "deaths"], axis = 1).rename( columns={ 'date': 'ds', 'deaths': 'y'})
# deaths_df
conf_df = df.reset_index().filter(["date", "confirmed"], axis = 1).rename( columns={ 'date': 'ds', 'confirmed': 'y'})
conf_df
mdl = fbprophet.Prophet( growth='linear', changepoint_prior_scale=0.98)
# mdl.add_changepoints_to_plot()
mdl.fit(conf_df)
forecast = mdl.make_future_dataframe(periods=4, freq='D')
forecast = mdl.predict(forecast)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat,yhat_lower,yhat_upper
67,2020-03-29,954.255676,909.192900,995.586166
68,2020-03-30,1010.466799,967.239565,1056.632004
69,2020-03-31,1069.648900,1029.023172,1114.674124
70,2020-04-01,1139.498350,1094.719281,1184.251985
71,2020-04-02,1205.333189,1159.621980,1256.739097


In [573]:
# fig = mdl.plot( forecast, xlabel= 'Date', ylabel='Confirmed')
fig = plot_plotly(mdl, forecast)  # This returns a plotly Figure
# a = add_changepoints_to_plot(fig.gca(), mdl, forecast)
plt.iplot(fig)
# forecast

In [0]:
forecast = forecast.set_index('ds') 
forecast = forecast[[ 'yhat', 'yhat_lower', 'yhat_upper']].apply(np.round)

In [575]:
df.index = pd.to_datetime(df.index)
forecast = pd.concat( [df.replace( np.nan, 0), forecast.replace( np.nan, 0)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [576]:
forecast

,confirmed,deaths,recovered,yhat,yhat_lower,yhat_upper
2020-01-22,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-23,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-24,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-25,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-26,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-03-29,NaN,NaN,NaN,954.0,909.0,996.0
2020-03-30,NaN,NaN,NaN,1010.0,967.0,1057.0
2020-03-31,NaN,NaN,NaN,1070.0,1029.0,1115.0
2020-04-01,NaN,NaN,NaN,1139.0,1095.0,1184.0


In [577]:
fig = go.Figure(data=[
                      go.Scatter(name='Upper limit', x=forecast.index, y=forecast['yhat_upper']),
                      go.Scatter(name='Confirmed Real', x=forecast.index, y=forecast['confirmed']),
                      go.Scatter(name='Confirmed Predicted', x=forecast.index, y=forecast['yhat']),
                      go.Scatter(name='Lower limit', x=forecast.index, y=forecast['yhat_lower'])
                      ])
fig.update_layout()
fig.show()